# CSC324 Lecture 3

**Lambda** is an anonymous function: If you want to use a function only once and you don't need to define it then you can use lambda. 
Examples: 
```
# Python
lambda x: x + 1

# Javascript: funcion (x) { return x + 1; }

# Haskell: \x > x+1
```

Functions: Consider the following function  
`f foo bar`. This is shorthand for `(f foo) bar` where bar is just optional.  
Similarly, for function types if we have a function of type `X -> Y -> A` then it is shorthand for `X -> (Y -> A)` 

In [1]:
-- Example for map
map f [1, 2, 3] = [f 1, f 2, f 3]

```
map (map f) [[1, 2 ,3], [4, 5 6]]
expands ... to
[[f 1, f 2, f 3], [f 4, f 5, f 6]]
```

Consider the following function

In [2]:
mySumV1 [] = 0
mySumV1 (x: xs) = x + mySumV1 xs

Line 1: Use foldr
Found:
mySumV1 [] = 0
mySumV1 (x : xs) = x + mySumV1 xs
Why not:
mySumV1 xs = foldr (+) 0 xs

In [3]:
myProductV1 [] = 1
myProductV1 (x:xs) = x * myProductV1 xs

Line 1: Use foldr
Found:
myProductV1 [] = 1
myProductV1 (x : xs) = x * myProductV1 xs
Why not:
myProductV1 xs = foldr (*) 1 xs

In [4]:
myMap f [] = []
myMap f (x:xs) = f x : myMap f xs

Line 1: Use map
Found:
myMap f [] = []
myMap f (x : xs) = f x : myMap f xs
Why not:
myMap f xs = map f xs

In [5]:
myProductV1 [1, 2, 3]
mySumV1 [1, 2, 3]

6

6

We could also write myMap as the following

In [6]:
myMap f lst = g lst 
    where 
        g [] = []
        g (x:xs) = f x : g xs

Line 1: Eta reduce
Found:
myMap f lst = g lst
Why not:
myMap f = gLine 3: Use map
Found:
g [] = []
g (x : xs) = f x : g xs
Why not:
g xs = map f xs

Notice that there are redundancies. Such functions have very common themes: 
```
f base case = solution to base case
f recurseive case (x: xs) = x `binary operator` f xs
```
### Foldr
All the above functions can be refactored to 
```
myFoldr op z lst = g lst
    where 
        g [] = z
        g (x:xs) = op x (g xs)
```
where `op` is the binary operation that you want to do, `z` is the solution to the base case and `lst` is your typical input 

In [7]:
myFoldr :: (a -> b -> b) -> b -> [a] -> b
myFoldr op z [] = z
myFoldr op z (x:xs) = op x (myFoldr op z xs)

In [8]:
mySumFoldr xs = foldr (+) 0 xs 
myProductFoldr xs = foldr (*) 1 xs
myMapFoldr f xs = foldr (:) [] xs

In [9]:
mySumFoldr [1, 2 ,15]

18

### How to detect if you have to use foldr

#### Base case 
If the base case involves an empty list or something of that sort. (Base case is some constant)

#### Induction step
Suppose the input has the form x:xs,  
Assume that your function works on xs  
You want to show for x:xs. 

### MyMap Example

#### Base case
`lst = []`  
then in this case we map [] to []
so `myMap f [] = []`

#### Induction step
Suppose myMap returns for xs. We want to show that `myMap f (x:xs) = foldr op [] (x:xs)`  
Expanding the left hand side, we get `f x : myMap f xs` when we manually write that code down  
However, we notice that it is equivalent to `op x (foldr op [] xs)`. By IH, this is `op x (myMap f xs)` and this is `myMap f (x:xs)`. 



Consider another way of summing the list. This is similar to how you write for loops

#### Base case 
`g accum []` in this case, it will just be `accum`

#### Induction step
Suppose you have a list (x:xs)  
We know by IH that xs works. i.e `g a xs = a + sum of xs`  

So now how do we compute `accum + sum of (x:xs)` ?  

```
accum + sum of (x:xs)
 = accum + x + sum of xs
 = (accum + x) + sum of xs
 = g (accum + x) xs               by IH
```
And so we're done. 

In [10]:
mySumV2 xs = g 0 xs -- for all a, xs: g a xs = a + sum of xs
    where
        g accum [] = accum
        
        
        g accum (x:xs) = g (accum + x) xs

Line 1: Eta reduce
Found:
mySumV2 xs = g 0 xs
Why not:
mySumV2 = g 0Line 3: Use foldl
Found:
g accum [] = accum
g accum (x : xs) = g (accum + x) xs
Why not:
g accum xs = foldl (+) accum xs

But we notice that this function has redundancies similar to the one discussed above. If we make other functions for `product`, `map` then we'll be repeating the same procedure.

Here is another example: Consider reversing a list

In [11]:
slowReverse [] = []
slowReverse (x:xs) = slowReverse xs ++ [x]

Note for the above function, this takes quadratic time as the `++` operatio
$$ \sum_{i} i = \frac{n(n+1)}{2} \in \mathcal{O}(n^2) $$


Consider reversing the list by using the accumulator.

`myReverse xs = g [] xs`

#### Induction Step
....
....
....


In [12]:
myReverse xs = g [] xs
    where
        g accum [] = accum
        g accum (x:xs) = g (x : accum) xs

Line 1: Eta reduce
Found:
myReverse xs = g [] xs
Why not:
myReverse = g []Line 3: Use foldl
Found:
g accum [] = accum
g accum (x : xs) = g (x : accum) xs
Why not:
g accum xs = foldl (flip (:)) accum xs

The common theme is the following:  
```
g accum [] = accum
g accum (x:xs) = g (a function of accum and x) xs
```

Example:  
```
mySumV2
initial accum: 0
function: (+)
```

But this does not help us resolve the issue of redundancy. In this case we can use foldl


In [13]:
myFoldl op z [] = z
myFoldl op z (x:xs) = myFoldl op (op z x) xs

mySumFoldl xs = myFoldl (+) 0 xs

myReverseFoldl xs = myFoldl (flip (:)) [] xs

In [14]:
mySumFoldl [1, 2, 3, 4]

10

In [15]:
myReverseFoldl [1, 2, 3 ,4]

[4,3,2,1]

You can think of foldl as expanding from the left hand side  
`foldl (+) 0 [1, 2, 3] = ((0 +1)+2)+3`

### Lazy Evaluation
Consider this messy function

In [16]:
take 0 _ = []
take _ [] = []
take n (x:xs) = x : take (n - 1) xs

In [17]:
take 2 [1, 2, 3, 4]

[1,2]

Typically, in static programming languages, if you define an infinite list and evaluate take function, it will never end. Surprisingly, Haskell does terminate even on infinite lists!

In [18]:
doITerminate = take 2 (foo 0)
  where
    foo n = n : foo (n + 1) -- foo = [1, 2, ...]

In [19]:
doIEvenMakeSense = take 2 foo
  where
    foo = 0 : foo -- foo = [0, 0, ...]

The idea is that Haskell does that following:
* Plug in the body and then evaluate that body. First do pattern matching and check if it matches the parameter
* If it is a match, call the function, otherwise check another pattern.
* Repeat step 1  

In this way, Haskell does not need to take the entire sequence into consideration!


#### How `doITerminate` works
```
take 2 (foo 0)
→ take 2 (0 : foo (0 + 1))
→ 0 : take (2-1) (foo (0 + 1))
→ 0 : take 1 (foo (0 + 1))
→ 0 : take 1 (n : foo (n + 1))          where n = 0 + 1
→ 0 : n : take (1-1) (foo (n + 1))
→ 0 : n : take (1-1) (foo (n + 1))      where n = 1
→ 0 : n : take 0 (foo (n + 1))      where n = 1
→ 0 : n : []                        where n = 1
```